In [1]:
import napari
import czifile
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
from monai.inferers import sliding_window_inference
from tnia.deeplearning.dl_helper import quantile_normalization
import torch.nn.functional as F
import tifffile
from tqdm import tqdm


raster_geometry not imported.  This is only needed for the ellipsoid rendering in apply_stardist


In [2]:
# Define the parent path and list of input CZI files
parent_path = Path(r'C:/Users/Alex/Desktop/Mailis')
input_files = sorted((parent_path / "data").glob("*.czi"))
print(f"Found {len(input_files)} files:")
for f in input_files:
    print(f.name)


Found 16 files:
M20E1-Stitching-08-Create Image Subset-01.czi
M20E2-Stitching-06-Create Image Subset-02.czi
M20E3-Stitching-05-Create Image Subset-03.czi
M20E4-Stitching-09-Create Image Subset-04.czi
M20E6-Stitching-07-Create Image Subset-07.czi
M20E6bis-Stitching-10-Create Image Subset-05.czi
M21E2-Stitching-01-Create Image Subset-08.czi
M21E3-Stitching-02-Create Image Subset-09.czi
M21E4-Stitching-03-Create Image Subset-10.czi
M21E5-Stitching-04-Create Image Subset-11.czi
M21E6-Stitching-05-Create Image Subset-12.czi
MockE1-Stitching-04-Create Image Subset-13.czi
MockE2-Stitching-01-Create Image Subset-14.czi
MockE3-Stitching-02-Create Image Subset-15.czi
MockE4-Stitching-03-Create Image Subset-16.czi
MockE5-Stitching-06-Create Image Subset-17.czi


In [3]:
# Load model
models_path = parent_path / 'models'
net = torch.load(Path(models_path) / 'test9.pth', weights_only=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)


In [4]:
# Prediction function returning class probabilities
def predict_probabilities(im, net):
    im = quantile_normalization(im)
    im = im.astype(np.float32)
    im_tensor = torch.from_numpy(im).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = sliding_window_inference(
            im_tensor,
            1024,
            1,
            net
        )

    probabilities = torch.nn.functional.softmax(logits, dim=1)
    return probabilities.squeeze(0).cpu().numpy()  # Shape: (num_classes, H, W)


In [5]:
# Choose class indices to save (can be one or more)
selected_classes = [0]  # Change this list to select desired classes (e.g., [0, 2])
print(f"Selected classes: {selected_classes}")


Selected classes: [0]


In [ ]:
# Save selected class probabilities as multi-channel or single-channel OME-TIFF
output_dir = parent_path / "image_predicted"
output_dir.mkdir(parents=True, exist_ok=True)

for czi_path in input_files:
    print(f"Processing: {czi_path.name}")
    image = czifile.imread(czi_path)
    image = np.squeeze(image)
    save_path = output_dir / (czi_path.stem + f"_classes{''.join(map(str, selected_classes))}_probabilities.ome.tif")

    # First slice prediction to determine dimensions
    first_probs = predict_probabilities(image[0, :, :], net)
    height, width = first_probs.shape[1:]
    depth = image.shape[0]
    num_selected = len(selected_classes)

    if num_selected == 1:
        all_probs = np.zeros((depth, height, width), dtype=np.float32)
        all_probs[0, :, :] = first_probs[selected_classes[0]]
    else:
        all_probs = np.zeros((num_selected, depth, height, width), dtype=np.float32)
        for c_idx, class_id in enumerate(selected_classes):
            all_probs[c_idx, 0, :, :] = first_probs[class_id]

    for i in tqdm(range(1, depth), desc=f"Predicting {czi_path.name}"):
        slice_probs = predict_probabilities(image[i, :, :], net)
        if num_selected == 1:
            all_probs[i, :, :] = slice_probs[selected_classes[0]]
        else:
            for c_idx, class_id in enumerate(selected_classes):
                all_probs[c_idx, i, :, :] = slice_probs[class_id]

    axes = 'ZYX' if num_selected == 1 else 'CZYX'
    tifffile.imwrite(
        save_path,
        all_probs,
        photometric='minisblack',
        ome=True,
        metadata={'axes': axes}
    )
    print(f"Saved probability map for classes {selected_classes} to {save_path}")


Processing: M20E1-Stitching-08-Create Image Subset-01.czi


Predicting M20E1-Stitching-08-Create Image Subset-01.czi: 100%|██████████████████████| 394/394 [09:41<00:00,  1.48s/it]


Saved probability map for classes [0] to C:\Users\Alex\Desktop\Mailis\image_predicted\M20E1-Stitching-08-Create Image Subset-01_classes0_probabilities.ome.tif
Processing: M20E2-Stitching-06-Create Image Subset-02.czi


Predicting M20E2-Stitching-06-Create Image Subset-02.czi: 100%|██████████████████████| 470/470 [05:47<00:00,  1.35it/s]


Saved probability map for classes [0] to C:\Users\Alex\Desktop\Mailis\image_predicted\M20E2-Stitching-06-Create Image Subset-02_classes0_probabilities.ome.tif
Processing: M20E3-Stitching-05-Create Image Subset-03.czi


...

Predicting M20E6-Stitching-07-Create Image Subset-07.czi: 100%|██████████████████████| 425/425 [06:10<00:00,  1.15it/s]


Saved probability map for classes [0] to C:\Users\Alex\Desktop\Mailis\image_predicted\M20E6-Stitching-07-Create Image Subset-07_classes0_probabilities.ome.tif
Processing: M20E6bis-Stitching-10-Create Image Subset-05.czi


Predicting M20E6bis-Stitching-10-Create Image Subset-05.czi: 100%|███████████████████| 425/425 [06:09<00:00,  1.15it/s]


Saved probability map for classes [0] to C:\Users\Alex\Desktop\Mailis\image_predicted\M20E6bis-Stitching-10-Create Image Subset-05_classes0_probabilities.ome.tif
Processing: M21E2-Stitching-01-Create Image Subset-08.czi


Predicting M21E2-Stitching-01-Create Image Subset-08.czi:  25%|██████████████▋                                           | 118/465 [01:49<05:31,  1.05it/s]

# 🔢 Save One or More Class Probability Maps

This notebook lets you select **one, two, or all three classes** to save their prediction probability maps from `.czi` image volumes.

## 🎛 How To Use:
- Edit the list `selected_classes` to match the class indices you want.
  - `[0]` for background only.
  - `[0, 2]` for class 0 and class 2.
  - `[0, 1, 2]` for all classes.

## 📂 Input Folder:
Place `.czi` files in:
```
C:/Users/Alex/Desktop/Mailis/data/
```

## 📁 Output Folder:
Saved predictions go to:
```
C:/Users/Alex/Desktop/Mailis/image_predicted/
```
